In [36]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
import h5py
from peakfinder import detect_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy as sp
from sklearn.metrics import classification_report
import similaritymeasures
from PIL import Image
from tqdm import tqdm_notebook as tqdm

In [37]:
#df_labels = pd.read_csv(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Scripts\smooth_labels.dat',delimiter = '\s+',header = None,index_col = None)
df_labels = pd.read_csv(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Scripts\modified_labels.txt',delimiter = '\s+',header = None,index_col = None)

#df_labels = pd.read_csv(r'/Users/kunal/OneDrive - The Open University/SPIN/Transmission/Scripts/smooth_labels.dat',delimiter = '\s+',header = None,index_col = None)

df_labels.columns = ['File','Label']

In [38]:
z = []
y = []

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

f_wav = 250

rootDir = r'C:\Users\kj4755\OneDrive - The Open University\SPIN\data\level_1p0_data\Occultation'
#rootDir = r'/Users/kunal/OneDrive - The Open University/SPIN/data/level_1p0_data/New occultations'
os.chdir(rootDir)
list_of_files = os.listdir(os.getcwd())

for each_file in list_of_files:
    for each_df_file in df_labels['File']:
        if each_file.startswith(each_df_file):
            
            file = h5py.File(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\data\level_1p0_data\Occultation\%s' %each_file,'r')
            #file = h5py.File(r'/Users/kunal/OneDrive - The Open University/SPIN/data/level_1p0_data/New occultations/%s' %each_file,'r')

            
            T = np.array(file['Science/Transmission'])
            TangAlt = np.array(file['Geometry/Point0/TangentAltSurface'])
            wav = np.array(file['Science/Wavelength'])

            avg_TangAlt = []

            for j in range(TangAlt.shape[0]):
                avg_TangAlt.append(np.mean(TangAlt[j,:]))



            T_wav = T[:,np.array(np.where(wav == find_nearest(wav,f_wav))).flatten()].reshape(-1,)

            if T_wav[0] > 0.5:
                T_wav = T_wav[::-1]

            z.append(T_wav)
            
            if avg_TangAlt[0] > 100:
                avg_TangAlt = avg_TangAlt[::-1]
            
            y.append(avg_TangAlt)

In [39]:
z = np.array(z)
y = np.array(y)

In [41]:
alt_interp = np.arange(0,274,1)
t_interp = []
for i in range(len(z)):
    z_interp = np.interp(alt_interp,y[i],z[i])
    t_interp.append(z_interp)
t_interp = np.array(t_interp)

In [42]:
t_clipped = []
for t in t_interp:
    clip = np.array([])
    peak_idx = detect_peaks(t) 
    for _ in t[peak_idx]:
        clip = np.append(clip,_)
        if _ > 0.98:
            break
    t_clipped.append(t[:np.where(t == clip[-1])[0][0]])
t_clipped = np.array(t_clipped)

In [43]:
clipped_len = []
for t in t_clipped: 
    clipped_len.append(len(t))
t_clipped_max = t_interp[:,:max(clipped_len)]

In [44]:
indices = np.arange(len(np.array(df_labels['Label'])))
#X_train, X_test, y_train, y_test,id_train,id_test = train_test_split(t_interp, np.array(df_labels['Label']),indices, test_size=0.33)
#X_train, X_test, y_train, y_test,id_train,id_test = train_test_split(t_clipped_max, np.array(df_labels['Label']),indices, test_size=0.33)

In [ ]:
#using clipped max
dm = np.zeros((len(X_test),len(X_train)))

for i in tqdm(range(len(X_test))):
    t0 = np.zeros((max(clipped_len),2))
    t0[:,0] = np.arange(max(clipped_len))
    t0[:,1] = X_test[i]
    t0t1 = np.zeros((max(clipped_len),2))
    for j in range(len(X_train)):
        t1 = np.zeros((max(clipped_len),2))
        t1[:,0] = np.arange(max(clipped_len))
        t1[:,1] = X_train[j]
        t0t1[:,0] = np.arange(max(clipped_len))
        t0t1[:,1] = X_test[i]/X_train[j]
        
        dm[i,j] =similaritymeasures.curve_length_measure(t0,t1)
        #dm[i,j] =similaritymeasures.dtw(t0,t1)[0]